In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
import os.path

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

import tensorflow as tf

from sklearn.metrics import confusion_matrix, classification_report

In [2]:
image_dir = Path(r"C:\Users\net pc\Desktop\THESIS PROJECT\AUGMENTED\DATA")

In [3]:
filepaths = list(image_dir.glob(r'*/*.JPG'))
labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1], filepaths))

filepaths = pd.Series(filepaths, name='Filepath').astype(str)
labels = pd.Series(labels, name='Label')

image_df = pd.concat([filepaths, labels], axis=1)

In [4]:
image_df

,Filepath,Label
0,C:\Users\net pc\Desktop\THESIS PROJECT\AUGMENT...,brain_tumor_augmented
1,C:\Users\net pc\Desktop\THESIS PROJECT\AUGMENT...,brain_tumor_augmented
2,C:\Users\net pc\Desktop\THESIS PROJECT\AUGMENT...,brain_tumor_augmented
3,C:\Users\net pc\Desktop\THESIS PROJECT\AUGMENT...,brain_tumor_augmented
4,C:\Users\net pc\Desktop\THESIS PROJECT\AUGMENT...,brain_tumor_augmented
...,...,...
39995,C:\Users\net pc\Desktop\THESIS PROJECT\AUGMENT...,kidney_tumor_augmented
39996,C:\Users\net pc\Desktop\THESIS PROJECT\AUGMENT...,kidney_tumor_augmented
39997,C:\Users\net pc\Desktop\THESIS PROJECT\AUGMENT...,kidney_tumor_augmented
39998,C:\Users\net pc\Desktop\THESIS PROJECT\AUGMENT...,kidney_tumor_augmented


In [5]:
train_df, test_df = train_test_split(image_df, train_size=0.80, shuffle=True, random_state=1)

In [6]:
from keras.preprocessing.image import ImageDataGenerator

train_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255,
                                                                  horizontal_flip=True,
                                                                  width_shift_range=0.2,
                                                                  height_shift_range=0.2,
                                                                  validation_split=0.2)

test_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

In [7]:
train_images = train_generator.flow_from_dataframe(dataframe=train_df,
                                                   x_col='Filepath',
                                                   y_col='Label',
                                                   target_size=(64, 64),
                                                   color_mode='rgb',
                                                   class_mode='binary',
                                                   batch_size=32,
                                                   shuffle=True,
                                                   seed=42,
                                                   subset='training')

val_images = train_generator.flow_from_dataframe(dataframe=train_df,
                                                 x_col='Filepath',
                                                 y_col='Label',
                                                 target_size=(64, 64),
                                                 color_mode='rgb',
                                                 class_mode='binary',
                                                 batch_size=32,
                                                 shuffle=True,
                                                 seed=42,
                                                 subset='validation')

test_images = test_generator.flow_from_dataframe(dataframe=test_df,
                                                 x_col='Filepath',
                                                 y_col='Label',
                                                 target_size=(64, 64),
                                                 color_mode='rgb',
                                                 class_mode='binary',
                                                 batch_size=32,
                                                 shuffle=False)

Found 25600 validated image filenames belonging to 2 classes.
Found 6400 validated image filenames belonging to 2 classes.
Found 8000 validated image filenames belonging to 2 classes.


In [8]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv3D, MaxPool3D , Dropout, Flatten
from tensorflow.keras.layers import Dropout

ResNet = tf.keras.applications.resnet50.ResNet50(
    include_top=False,
    weights='imagenet',
    input_shape=(64, 64, 3))


#ResNet = ResNet50(input_shape= (64, 64, 3), weights='imagenet', include_top=False)
ResNet.trainable = False

In [9]:
model = keras.Sequential([
    ResNet,
    keras.layers.Flatten(),
#    keras.layers.Dropout(0.5),
#    keras.layers.Dense(units=32, activation='relu'),
#    keras.layers.Dropout(0.25),
    keras.layers.Dense(units=1, activation='sigmoid'),
])

In [10]:
model.compile(
  loss='binary_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 2, 2, 2048)        23587712  
                                                                 
 flatten (Flatten)           (None, 8192)              0         
                                                                 
 dense (Dense)               (None, 1)                 8193      
                                                                 
Total params: 23595905 (90.01 MB)
Trainable params: 8193 (32.00 KB)
Non-trainable params: 23587712 (89.98 MB)
_________________________________________________________________


In [12]:
for layer in model.layers:
    print(layer, layer.trainable)

<keras.src.engine.functional.Functional object at 0x0000025315C87AD0> False
<keras.src.layers.reshaping.flatten.Flatten object at 0x00000253151523D0> True
<keras.src.layers.core.dense.Dense object at 0x0000025315C98C90> True


In [ ]:
history = model.fit(
    train_images,
    validation_data=val_images,
    epochs=50,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=5,
            restore_best_weights=True
        ),
        tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            patience=3
        )
    ]
)

Epoch 1/50


800/800 [==============================] - 141s 174ms/step - loss: 0.3939 - accuracy: 0.8396 - val_loss: 0.3196 - val_accuracy: 0.8725 - lr: 0.0010
Epoch 2/50
800/800 [==============================] - 245s 306ms/step - loss: 0.3005 - accuracy: 0.8770 - val_loss: 0.2836 - val_accuracy: 0.8938 - lr: 0.0010
Epoch 3/50
800/800 [==============================] - 238s 297ms/step - loss: 0.2662 - accuracy: 0.8921 - val_loss: 0.2615 - val_accuracy: 0.8916 - lr: 0.0010
Epoch 4/50
800/800 [==============================] - 242s 302ms/step - loss: 0.2478 - accuracy: 0.9000 - val_loss: 0.2457 - val_accuracy: 0.9039 - lr: 0.0010
Epoch 5/50
800/800 [==============================] - 238s 297ms/step - loss: 0.2411 - accuracy: 0.9036 - val_loss: 0.2401 - val_accuracy: 0.9017 - lr: 0.0010
Epoch 6/50
800/800 [==============================] - 236s 295ms/step - loss: 0.2353 - accuracy: 0.9054 - val_loss: 0.2274 - val_accuracy: 0.9119 - lr: 0.0010
Epoch 7/50
800/800 [========================

In [ ]:
import matplotlib.pyplot as plt

#plt.figure(figsize=(14, 10))
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('VGG16 Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Accuracy', 'Validation Accuracy', 'loss', 'Validation Loss'])
plt.show()

In [ ]:
import matplotlib.pyplot as plt

#plt.figure(figsize=(14, 10))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('VGG16 Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper right')
plt.show()

In [ ]:
results = model.evaluate(test_images, verbose=0)

print("    Test Loss: {:.5f}".format(results[0]))
print("Test Accuracy: {:.2f}%".format(results[1] * 100))

In [ ]:
predictions = (model.predict(test_images) >= 0.5).astype(int)

cm = confusion_matrix(test_images.labels, predictions, labels=[0, 1])
clr = classification_report(test_images.labels, predictions, labels=[0, 1], target_names=["Brain Tumor", "Kidney tumor"])

plt.figure(figsize=(6, 6))
sns.heatmap(cm, annot=True, fmt='g', vmin=0, cmap='Blues', cbar=False)
plt.xticks(ticks=[0.5, 1.5], labels=["Brain Tumor", "Kidney tumor"])
plt.yticks(ticks=[0.5, 1.5], labels=["Brain Tumor", "Kidney tumor"])
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix of VGG16 Model")
plt.show()

print("Classification Report:\n----------------------\n", clr)

In [ ]:
# Obtain predicted probabilities
from sklearn.metrics import roc_auc_score, roc_curve

predicted_probabilities = model.predict(test_images)

# Compute AUC ROC
auc_roc = roc_auc_score(test_images.labels, predicted_probabilities)

print("AUC ROC: {:.5f}".format(auc_roc))

# Plot ROC curve
fpr, tpr, thresholds = roc_curve(test_images.labels, predicted_probabilities)

plt.figure()
plt.plot(fpr, tpr, label='ROC curve (area = {:.2f})'.format(auc_roc))
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.show()

In [ ]:
# save the model to a file
model.save('ResNet50-Brain Tumor vs Kidney tumor .h5')